In [1]:
import os
import pynq
import numpy as np
# os.environ['EMCONFIG_PATH'] = os.environ['PWD']
ol=pynq.Overlay("krnl_matmulbertl_opt.xclbin")


/data/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
from pynq import allocate
Nbanks=8
Nmat=1
Tsize=1024
Nvec=14

In [3]:
# low = 1 # -2^15
# high =  200# 2^15-1
low = -100
high = 100
source_v_np = np.random.randint(low, high, dtype=np.int8, size=(Tsize,Nvec))
source_v_np.shape

(1024, 14)

In [4]:
low = -100
high = 100
source_w_np = np.random.randint(low, high, dtype=np.int8, size=(Nmat*Tsize,Tsize))
source_w_np[100,13] = 2
source_w_np.shape


(1024, 1024)

In [5]:
source_w_split_np = []
for i in range(Nbanks):
    source_w_split_np.append(np.zeros((Nmat*Tsize,Tsize//Nbanks)))

In [6]:
source_w_split_np[0].shape

(1024, 128)

In [7]:
# save the split matrices of the reference values for the loading
source_w_split_np = np.hsplit(source_w_np, Nbanks)
source_w_split_np[0].shape

(1024, 128)

In [8]:
outbuf_np = np.zeros((Nvec, Tsize), dtype=np.int32)
outbuf_np.shape

(14, 1024)

In [9]:
outbuf_np = np.matmul(source_w_np, source_v_np, dtype=np.int32)
outbuf_np, outbuf_np.shape

(array([[   -121,   11482, -108132, ...,   31808,   26676,   19100],
        [ -22933,   82349, -230029, ...,  -14270,  -17445,  -32602],
        [  97154,   64067,  -15278, ...,   26689,  165909, -128912],
        ...,
        [  13098,  222216,  120247, ...,  -16820,  -46254,  -59414],
        [ 180275,  -32939,  -64279, ...,  192072,  -95258,   -5030],
        [ -69859,  -52796,  -34661, ..., -175254,   75385,   59810]],
       dtype=int32),
 (1024, 14))

In [26]:

%%timeit 
# w alloc
source_w = [
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM0),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM4),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM8),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM12),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM16),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM20),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM24),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM26)]
 
for i in range(len(source_w)):
#     source_w[i][:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int8, size=(Nmat*Tsize//Nbanks,Tsize))
    source_w[i][:] = source_w_split_np[i]
# nmat, nvec
# 1, 14: 6.51 ms ± 553 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


6.51 ms ± 553 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
source_w[0].shape, source_w_split_np[0].shape

((1024, 128), (1024, 128))

In [13]:
source_w[0].shape


(1024, 128)

In [27]:
%%timeit

# w sync
for i in range(8):
    source_w[i].sync_to_device()
    
# nmat, nvec
# 1, 14: 355 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


355 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
%%timeit

# v alloc and prep
source_v = pynq.allocate(shape=(Nvec,Tsize), dtype=np.int8, target=ol.HBM14)

source_v[:] = source_v_np.T

# nmat, nvec
# 1, 14: 806 µs ± 59.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


806 µs ± 59.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%%timeit

# out alloc

outbuf = pynq.allocate((Tsize*Nmat,Nvec), dtype=np.int32, target=ol.HBM14)

# nmat, nvec
# 1, 14: 803 µs ± 77.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

803 µs ± 77.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%%timeit

# v sync
source_v.sync_to_device()

# nmat, nvec
# 1, 14: 23.9 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

23.9 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
print("source_v shape", source_v.shape)
print(type(source_v))
print("outbuf shape", outbuf.shape)


In [32]:
%%timeit
ol.feeder_1.call(
	source_v,
	source_w[0],
	source_w[1],
	source_w[2],
	source_w[3],
	source_w[4],
	source_w[5],
	source_w[6],
	source_w[7],
    outbuf, 
    Nmat,
    Nvec, # seq length
    0)

outbuf.sync_from_device()
# nmat, nvec
# 1, 14: 1.16 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 1, 128: 1.67 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 3, 14 : 1.31 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 3, 128: 3.32 ms ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 4, 128: 3.82 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 
# 8, 128: 6.87 ms ± 77.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


1.17 ms ± 99.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
outbuf.shape

(1024, 14)

In [18]:
outbuf_np.shape

(1024, 14)

In [19]:
source_v

PynqBuffer([[-88,  71, -76, ..., -91, -29, -37],
            [ 66,  95,  13, ...,  31,  35, -11],
            [-35,  30,  80, ..., -48, -72, -34],
            ...,
            [ 34, -70,  76, ...,   0, -65, -64],
            [-55, -77,  54, ...,  45,  91, -34],
            [ 47, -75,  91, ...,  89, -61,  95]], dtype=int8)

In [20]:
source_w[0]

PynqBuffer([[ 46, -78, -92, ...,  73,  37,  43],
            [-27,   2,  93, ...,  59,  23, -80],
            [ 21, -93, -47, ...,  47, -12,  66],
            ...,
            [-45, -98,  13, ...,  94, -14,  81],
            [-61, -75,  77, ...,  93,  32, -50],
            [ -9, -38,  22, ...,  69, -27, -23]], dtype=int8)

In [21]:
outbuf

PynqBuffer([[   -121,   11482, -108132, ...,   31808,   26676,   19100],
            [ -22933,   82349, -230029, ...,  -14270,  -17445,  -32602],
            [  97154,   64067,  -15278, ...,   26689,  165909, -128912],
            ...,
            [  13098,  222216,  120247, ...,  -16820,  -46254,  -59414],
            [ 180275,  -32939,  -64279, ...,  192072,  -95258,   -5030],
            [ -69859,  -52796,  -34661, ..., -175254,   75385,   59810]],
           dtype=int32)

In [22]:
outbuf_np

array([[   -121,   11482, -108132, ...,   31808,   26676,   19100],
       [ -22933,   82349, -230029, ...,  -14270,  -17445,  -32602],
       [  97154,   64067,  -15278, ...,   26689,  165909, -128912],
       ...,
       [  13098,  222216,  120247, ...,  -16820,  -46254,  -59414],
       [ 180275,  -32939,  -64279, ...,  192072,  -95258,   -5030],
       [ -69859,  -52796,  -34661, ..., -175254,   75385,   59810]],
      dtype=int32)

In [23]:
outbuf.any() == outbuf_np.any()

PynqBuffer(True)